In [3]:
#Imports
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [85]:
# Reading GLCM unclean
df_train =pd.read_csv('/content/drive/MyDrive/GLCM/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/GLCM/test.csv')
df_val = pd.read_csv('/content/drive/MyDrive/GLCM/val.csv')

In [86]:
display(df_train)

,Patient ID,Slice ID,Lung Volume,Angular_Second_Momentum,Contrast,Correlation,Entropy
0,ID00007637202177411956430,1,0,NaN,NaN,NaN,NaN
1,ID00007637202177411956430,10,45163,0.002000,63.558,0.002000,7.126
2,ID00007637202177411956430,11,48264,0.002000,67.037,0.002000,7.179
3,ID00007637202177411956430,12,49509,0.001000,61.217,0.002000,7.278
4,ID00007637202177411956430,13,51046,0.001000,68.085,0.002000,7.340
...,...,...,...,...,...,...,...
4333,ID00426637202313170790466,392,0,NaN,NaN,NaN,NaN
4334,ID00426637202313170790466,403,0,NaN,NaN,NaN,NaN
4335,ID00426637202313170790466,48,2888,0.000545,160.860,0.001000,7.874
4336,ID00426637202313170790466,6,0,NaN,NaN,NaN,NaN


In [87]:
# Make Slice Id to go from 0 to 29 
def slice_proper(df):
  df_clean=pd.DataFrame(columns = ["Patient ID","Slice ID","Lung Volume","Angular_Second_Momentum","Contrast","Correlation","Entropy","New_Slice"])
  Pat_unique=df['Patient ID'].unique()
  for pat in Pat_unique:
    r_df = df[df['Patient ID'] == pat] 
    sorted_r=r_df.sort_values(by=['Slice ID'])
    new_slice=[i for i in range(len(sorted_r))]
    sorted_r["New_Slice"]=new_slice
    df_clean=df_clean.append(sorted_r)
  return df_clean

In [88]:
#Cleaning
c_df_train=slice_proper(df_train)
c_df_test=slice_proper(df_test)
c_df_val=slice_proper(df_val)
print(c_df_val)

                    Patient ID Slice ID Lung Volume  Angular_Second_Momentum  \
0    ID00019637202178323708467        1           0                      NaN   
11   ID00019637202178323708467        2           0                      NaN   
22   ID00019637202178323708467        3           0                      NaN   
23   ID00019637202178323708467        4        6636                 0.000701   
24   ID00019637202178323708467        5       18246                 0.000784   
..                         ...      ...         ...                      ...   
264  ID00119637202215426335765       55        3464                 0.000674   
265  ID00119637202215426335765       57        1298                 0.000803   
266  ID00119637202215426335765       59         210                 0.001000   
267  ID00119637202215426335765       60           0                      NaN   
268  ID00119637202215426335765       62           0                      NaN   

     Contrast  Correlation  Entropy New

In [102]:
def Create_clean_data_frame(df):
  df_GLCM_ALL=pd.DataFrame(columns = ["Patient","Lung Volume","T_ASM","T_Contrast","T_Correlation","T_Entropy","M_ASM","M_Contrast","M_Correlation","M_Entropy","B_ASM","B_Contrast","B_Correlation","B_Entropy"])
  Pat_unique=df['Patient ID'].unique()
  for kkk,pat in enumerate(Pat_unique):
    r_df = df[df['Patient ID'] == pat] 
    column = r_df["Lung Volume"]
    max_value_Lung_Volume = column.max()
    #print(max_value_Lung_Volume)
    rdf_len=len(r_df)
    
    # First partition - Top 
    top=r_df[r_df['New_Slice'] <= rdf_len/3]
    top.dropna(inplace=True)
    top_ASM=top['Angular_Second_Momentum'].mean()
    top_Contrast=top['Contrast'].mean()
    top_Correlation=top['Correlation'].mean()
    top_Entropy=top['Entropy'].mean()
   

    #Middle
    Mid=r_df[(r_df['New_Slice'] <= 2*rdf_len/3) & (r_df['New_Slice'] > rdf_len/3)]
    Mid.dropna(inplace=True)
    Mid_ASM=Mid['Angular_Second_Momentum'].mean()
    Mid_Contrast=Mid['Contrast'].mean()
    Mid_Correlation=Mid['Correlation'].mean()
    Mid_Entropy=Mid['Entropy'].mean()

    #Bottom
    Bot=r_df[r_df['New_Slice'] > 2*rdf_len/3]
    Bot.dropna(inplace=True)
    Bot_ASM=Bot['Angular_Second_Momentum'].mean()
    Bot_Contrast=Bot['Contrast'].mean()
    Bot_Correlation=Bot['Correlation'].mean()
    Bot_Entropy=Bot['Entropy'].mean()
    df_GLCM_ALL.loc[len(df_GLCM_ALL.index)]=[pat,max_value_Lung_Volume,top_ASM,top_Contrast,top_Correlation,top_Entropy,Mid_ASM,Mid_Contrast,Mid_Correlation,Mid_Entropy,Bot_ASM,Bot_Contrast,Bot_Correlation,Bot_Entropy]
  return df_GLCM_ALL
    

In [103]:
df_train_GLCM=Create_clean_data_frame(c_df_train)
df_test_GLCM=Create_clean_data_frame(c_df_test)
df_val_GLCM=Create_clean_data_frame(c_df_val)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [104]:
print(len(df_test_GLCM))

17


In [105]:
print(df_test_GLCM)

                      Patient Lung Volume     T_ASM  T_Contrast  \
0   ID00010637202177584971671       51270  0.000310  322.540667   
1   ID00025637202179541264076      136598  0.003750   37.176250   
2   ID00038637202182690843176      145693  0.002579   47.430000   
3   ID00061637202188184085559       56571  0.001735   94.796125   
4   ID00077637202199102000916       72085  0.000563  147.049625   
5   ID00082637202201836229724      121449  0.003889   28.581333   
6   ID00089637202204675567570      160140  0.001214  172.583625   
7   ID00104637202208063407045       47612  0.003200   39.447900   
8   ID00123637202217151272140       63039  0.000614  965.592000   
9   ID00125637202218590429387       44551  0.001667   90.731833   
10  ID00128637202219474716089         330  0.163750  235.574000   
11  ID00131637202220424084844       51510  0.000603  154.238778   
12  ID00135637202224630271439       70847  0.000597  203.625000   
13  ID00139637202231703564336       70263  0.006000   42.78977

In [106]:
df_test_GLCM.to_csv("/content/drive/MyDrive/GLCM_test.csv")
df_train_GLCM.to_csv("/content/drive/MyDrive/GLCM_train.csv")
df_val_GLCM.to_csv("/content/drive/MyDrive/GLCM_val.csv")